## Important Dependencies with their required versions


In [4]:

# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb


## Imports and Setup and Initializing Agent

In [5]:
from Imports import *
import sys
from io import StringIO
def runCode(code):
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    oldstderr = sys.stderr
    redirected_output2 = sys.stderr = StringIO()
    result = ""
    thread = CustomThread()
    try:
        thread.start()
        exec(code, globals())
        result = redirected_output.getvalue()
    except Exception as e:
        # print(repr(e))
        result = repr(e)
    except SystemExit as s:
        # print(repr(s))
        result = redirected_output2.getvalue()
    except KeyboardInterrupt as k:
        result = "timed out"
    thread.stop()
    globals().popitem()
    sys.stdout = old_stdout
    sys.stderr = oldstderr
    # print(result)
    return result
#########################


# llm2,llm3=InitializeGptModel(os.environ['OPENAI_API_KEY'])


## Generate Unit Tests from Code

In [3]:
# from Imports import *
try:
    from Configuration import *
    from TestGenerator import *
    from TestFix import *
    t = TestGenerator(GenUnitTestChain, db, HEval_JsonObj, globals())
    f = TestFix(globals())
except:
    print("Unable to connect to HuggingFace API")

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


In [2]:
t.generate()

Running Test Case  0
FAIL: test_has_close_elements_first_and_last_close (__main__.TestHasCloseElements)
Traceback (most recent call last):
  File "<string>", line 40, in test_has_close_elements_first_and_last_close
AssertionError: False is not true


Test example 0 failed

Number of Ran Tests :  7
Number of failed Tests :  1
Number of error Tests :  0
Number of Succeeded Test :  6
Running Test Case  1

Test example 1 succeeded

Number of Ran Tests :  7
Number of Succeeded Test :  7
Number of Succeeded Test :  0
Running Test Case  2
FAIL: test_truncate_number_6 (__main__.TestTruncateNumber)
Traceback (most recent call last):
  File "<string>", line 33, in test_truncate_number_6
AssertionError: 0.77 != 0.23 within 7 places (0.54 difference)


Test example 2 failed

Number of Ran Tests :  7
Number of failed Tests :  1
Number of error Tests :  0
Number of Succeeded Test :  6
Running Test Case  3

Test example 3 succeeded

Number of Ran Tests :  7
Number of Succeeded Test :  7
Number of Suc

In [2]:
# from TestFix import *
# f = TestFix(globals())

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


In [4]:
f.generate()

Running Example  0 

original feedback is:  ...F...
FAIL: test_has_close_elements_first_and_last_close (__main__.TestHasCloseElements)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<string>", line 40, in test_has_close_elements_first_and_last_close
AssertionError: False is not true

----------------------------------------------------------------------
Ran 7 tests in 0.002s

FAILED (failures=1)

Test example 0 failed

Number of Ran Tests :  7
Number of failed Tests :  1
Number of error Tests :  0
Number of Succeeded Test :  6
Old Error Tests 
Running Example  1 

Example 1  has already passed
Running Example  2 

original feedback is:  .....F.
FAIL: test_truncate_number_6 (__main__.TestTruncateNumber)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<string>", line 33, in test_truncate_number_6
AssertionError: 0.77 != 0.23 within 7 places (0.54 difference